In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [2]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, status, _ = fbg.eval_global_conn_k(states, G_base, target_k=2)
print("k =", k_val, "status =", status)

k = 1 status = f


In [3]:
row_names = list(edges.keys()) + ['sys']
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [4]:
def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = 0.5, # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st == 's':
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', 1) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

In [5]:
origin = 'n1'
dests = ['n22', 'n66']

In [6]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / "rules_surv.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / "rules_fail.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

C:\Users\jb622s\AppData\Local\Temp\ipykernel_24560\2870499433.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(TSUMPATH / "rules_surv.pt", map

# Calculate system probabilities

## Marginal probability

In [7]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun
)
print(f"P(sys=1) = {pr_cond['survival']:.3e}")
print(f"P(sys=0 ) = {pr_cond['failure']:.3e}\n")

P(sys=1) = 9.999e-01
P(sys=0 ) = 1.130e-04



## Conditional probability given "one" component's survival

In [ ]:
for x in row_names[:-1]:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun
    )
    print(f"P(sys=1 | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys=0 | {x}=1) = {pr_cond['failure']:.3e}\n")

Eval P(sys | e0001=1)
P(sys=1 | e0001=1) = 9.999e-01
P(sys=0 | e0001=1) = 1.060e-04

Eval P(sys | e0002=1)
P(sys=1 | e0002=1) = 9.999e-01
P(sys=0 | e0002=1) = 9.900e-05

Eval P(sys | e0003=1)
P(sys=1 | e0003=1) = 9.999e-01
P(sys=0 | e0003=1) = 1.340e-04

Eval P(sys | e0004=1)
P(sys=1 | e0004=1) = 9.999e-01
P(sys=0 | e0004=1) = 1.130e-04

Eval P(sys | e0005=1)
P(sys=1 | e0005=1) = 9.999e-01
P(sys=0 | e0005=1) = 9.500e-05

Eval P(sys | e0006=1)
P(sys=1 | e0006=1) = 9.999e-01
P(sys=0 | e0006=1) = 1.120e-04

Eval P(sys | e0007=1)
P(sys=1 | e0007=1) = 9.999e-01
P(sys=0 | e0007=1) = 1.090e-04

Eval P(sys | e0008=1)
